_Cell 1: jupyter helpers_

In [1]:
# jupyter helpers

# To get multiple outputs from one cell:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
from IPython.display import HTML, Markdown, Image
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))


def add_div(div_class='info', div_start='Tip:', 
            div_text='Some tip here', output_string=True):
    """
    Behaviour with default `output_string=True`:
    The cell is overwritten with the output, but the cell mode is still 'code',
    not 'markdown'.
    Workaround: After running the function, click on the new cell, press ESC, 
                type 'm', then run the new cell.
    If `output_string=False`, the output is displayed in an new cell with the 
    code cell visible.
    ```
    [x]
    add_div('alert-warning', 'Tip: ', 'some tip here', output_string=True)
    [x]
    <div class="alert alert-warning"><b>Tip: </b>some tip here</div>
    ```
    """
    accepted = ['info', 'warning', 'danger']
    div_class = div_class.lower()
    if div_class not in accepted:
        msg = f'<div class="alert"><b>Wrong class:&nbsp;</b> `div_start` not in: {accepted}.</div>'
        return Markdown(msg)
    
    div = f"""<div class="alert alert-{div_class}"><b>{div_start}&nbsp;&nbsp;</b>{div_text}</div>"""
    if output_string:
        return get_ipython().set_next_input(div, 'markdown')
    else:
        return Markdown(div)

    
def new_section(title='New section'):
    style = "text-align:center;background:#c2d3ef;padding:16px;color:#ffffff;font-size:2em;width:98%"
    div = f'<div style="{style}">{title}</div>'
    #return HTML('<div style="{}">{}</div>'.format(style, title))
    return get_ipython().set_next_input(div, 'markdown')


_Cell 2: project helpers_

In [23]:
# project helpers

import sys
from pathlib import Path
from pprint import pprint as pp

def sys_info():
    frmt = "\nListing from sys_info():\n"
    frmt += "Python ver: {}\nPython env: {}\n"
    frmt += "OS:         {}\nCurrent dir: {}\n"
    print(frmt.format(sys.version, 
                      Path(sys.prefix).name,
                      sys.platform,
                      Path.cwd()))


def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """
    newp = Path(this_path).as_posix() # no str method (?)
    if up:
        newp = Path(this_path).parent.as_posix()

    msg = 'Path already in sys.path'
    if newp not in sys.path:
        sys.path.insert(1, newp)
        msg = 'Path added to sys.path'
    print(msg)

# if notebook inside another folder, eg ./notebooks:
nb_folder = 'notebooks'
add_to_sys_path(Path.cwd(), Path.cwd().name.startswith(nb_folder))


# Filtered dir() for method discovery:
def filter_dir(mdl, filter_str=None, start_with_str='_', exclude=True):
    """Filter dir(mdl) for method discovery.
       Input:
       :param mdl (object): module, optionally with submodule path(s), e.g. mdl.submdl1.submdl2.
       :param filter_str (str, None): filter all method names containing that string.
       :param start_with_str (str, '_'), exclude (bool, True): start_with_str and exclude work 
              together to perform search on non-dunder methods (default).
    """
    search_dir = [d for d in dir(mdl) if not d.startswith(start_with_str) == exclude]
    if filter_str is None:
        return search_dir
    else:
        filter_str = filter_str.lower()
        return [d for d in search_dir if d.lower().find(filter_str) != -1]


def get_project_dirs(which: list,
                     nb_folder='notebooks',
                     use_parent=True):
    '''Create folder(s) named in `which` at the parent level.'''
    dir_lst = []
    if Path.cwd().name.startswith(nb_folder) or use_parent:
        dir_fn = Path.cwd().parent.joinpath
    else:
        dir_fn = Path.cwd().joinpath
        
    for d in which:
        DIR = dir_fn(d)
        if not DIR.exists():
            Path.mkdir(DIR)
        dir_lst.append(DIR)
    return dir_lst

#DIR_DATA, 
DIR_IMG, DIR_TMP = get_project_dirs(['images', '_temp'])


# autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    get_ipython().run_line_magic('load_ext', 'autoreload')

%autoreload 2

#..................
sys_info()

no_wmark = False
try:
    %load_ext watermark
    #%watermark
except ModuleNotFoundError:
    no_wmark = True

if not no_wmark:
    print("\nListing from watermark ext:")
    %watermark
    #-iv

Path already in sys.path
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Listing from sys_info():
Python ver: 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:16:33) [MSC v.1929 64 bit (AMD64)]
Python env: miniconda3
OS:         win32
Current dir: C:\Users\catch\Documents\GitHub\new_conda_env\notebooks

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark

Listing from watermark ext:
Last updated: 2023-02-05T16:59:05.121346-05:00

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.9.0

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 142 Stepping 10, GenuineIntel
CPU cores   : 8
Architecture: 64bit



---
---
# Testing modules
---

In [3]:
from new_conda_env import main

In [5]:
# name of env to 'quick-clone':
env_to_clone = 'ds310'

# remove periods in py ver if True:
dotless_ver = False

old_py_ver = '3.10'   # needed to check if different from new?
new_py_ver = '3.11'

conda_vir = main.CondaEnvir(old_py_ver, new_py_ver, dotless_ver, env_to_clone)
conda_vir
print(conda_vir)

CondaEnvir(3.10 (str), 3.11 (str), False (bool), ds310 (str), None (new_env_name='default', str))

DOC: CondaEnvir is a class that gathers basic information
    about the current conda framework in order to perform the
    'quick-clone'* of a conda environment when, e.g. a new 
    version of the (python) kernel is needed.
    [* see README.md]
    
    Arguments:
    - old_py_ver (str): Previous python version
    - new_py_ver (str): New python version
    - dotless_ver (bool): If True, period(s) in new_py_ver are removed when forming
      the default `new_env_name`
    - env_to_clone (str): The exisitng conda environment to 'quick-clone'
    - new_env_name (str, "default"): If called with default value, the new env name will
      be autogenerated as such: 'env' + self.new_py_ver
    - debug (bool, True): flag for tracing calls.
    


In [27]:
filter_dir(main)

['CondaEnvir',
 'PIPE',
 'Path',
 'Popen',
 'base_csts',
 'context',
 'functools',
 'getLogger',
 'load_env_yml',
 'log',
 'os',
 'subprocess',
 'sys',
 'sys_rc_path',
 'user_rc_path',
 'yaml',
 'yaml_round_trip_dump',
 'yaml_round_trip_load']

In [6]:
filter_dir(conda_vir)

['basic_info',
 'conda_root',
 'debug',
 'dotless_ver',
 'env_to_clone',
 'get_conda_info',
 'get_new_env_name',
 'get_new_prefix',
 'get_user_rc',
 'has_user_rc',
 'new_env_name',
 'new_py_ver',
 'old_py_ver',
 'run_conda_export',
 'same_vers',
 'user_rc']

In [11]:
if conda_vir.basic_info['active_prefix'] != conda_vir.conda_root:
    print("`new_cond_env` should be run in (base), but this environment is activated:",
          conda_vir.basic_info['active_prefix'].name)

`new_cond_env` should be run in (base), but this environment is activated: ds310


In [33]:
test_yml_nobld = "env_ds310_nobld.yml"
test_yml_hist = "env_ds310_hist.yml"

yml_nobld_path = DIR_TMP.joinpath(test_yml_nobld)
yml_nobld_path.exists()

yml_hist_path = DIR_TMP.joinpath(test_yml_hist)
yml_hist_path.exists()

yml_nobld = main.load_env_yml(yml_nobld_path)
yml_hist = main.load_env_yml(yml_hist_path)
# keys :: ['name', 'channels', 'dependencies', 'prefix']

True

True

In [47]:
# retrieve pip dict from "long yml" file:

pip_deps = [v for v in yml_nobld['dependencies'] if isinstance(v, dict)][0]
if pip_deps:
    # make sure the dict is for pip (?)
    if pip_deps.get('pip') is None:
        pip_deps = None
    else:
        #pp(pip_deps)
        # yaml-dump to str:
        yml_deps = main.yaml_round_trip_dump(pip_deps)  # type(yml_deps) : str
        
yml_deps
        
# remove versions
import re

regex = r"(?<=)==\d+(?:\.\d+){2}"
cleaned_deps = re.sub(regex, "", yml_deps)
cleaned_deps

'pip:\n  - actdiag==3.0.0\n  - blockdiag==3.0.0\n  - blockdiagmagic==0.0.3\n  - funcparserlib==1.0.1\n  - matplotlib-venn==0.11.7\n  - networkx==3.0\n  - nwdiag==3.0.0\n  - seqdiag==3.0.0\n  - watermark==2.3.1\n  - webcolors==1.12\n'

'pip:\n  - actdiag\n  - blockdiag\n  - blockdiagmagic\n  - funcparserlib\n  - matplotlib-venn\n  - networkx==3.0\n  - nwdiag\n  - seqdiag\n  - watermark\n  - webcolors==1.12\n'

In [50]:
print(yml_hist['dependencies'], type(yml_hist['dependencies']), sep='\n')

['flake8', 'beautifulsoup4', 'matplotlib', 'python=3.10', 'seaborn', 'pandas', 'ipython', 'python-dotenv', 'black', 'python-graphviz', 'ipykernel', 'pygraphviz', 'scipy', 'ipywidgets', 'numpy', 'pytest', 'pylint', 'scikit-learn', 'dask', 'pyspark', 'pyarrow', 'defusedxml']
<class 'ruamel.yaml.comments.CommentedSeq'>


In [52]:
yml_hist['dependencies'].append

Object `append` not found.


In [135]:
import os
from pathlib import Path
import subprocess
from subprocess import Popen, PIPE

import yaml
from yaml import Loader, Dumper


def get_yml_file(yml_path: Path) -> dict:
    if yml_path.exists():
        with open(yml_path) as stream:
            yml_file = yaml.load(stream, Loader)
        return yml_file
    raise FileNotFoundError(f"NotFound: {yml_path}")
    
    
def save_dict_to_yml(yml_filepath: Path, yml_dict: dict) -> None:
    with open(yml_filepath, 'w') as out:
        yaml_round_trip_dump(yml_dict, out)



def get_python_new_deps(env_file:dict, new_py_ver: str) -> list:
    """Add or amend the yaml dependencies for python.
    """
    
    deps = env_file["dependencies"]
    new_deps = []
    pys = [i for i, d in enumerate(deps) if d.startswith("python=")]
    if pys:
        del deps[pys[0]]  # in case there are several, possible??
   
    return ["python="+new_py_ver] + deps 


def create_new_env_yml(user_dir, new_py_ver, new_env_name):
    """
    """
    
def main() -> list:
    new_py_ver = "3.9"
    new_env_name = "fav39"
    
    # For testing, env_generic_minimal_test.yml has no python dep & prefix line is "prefix: envs_path\some_name"
    yml_filename = "env_generic_minimal_test.yml"
    yml_new_filename = yml_filename.replace("test", new_env_name)
    
    # todo: run conda export --from history > yml_filename if not .exists()

    python_dirs = get_python_dirs().split('\n')
    user_dirs = get_user_dirs(python_dirs)
    user_dir = user_dirs[0]                    # consider 1st one only
    yml_path = user_dir.joinpath(yml_filename) 
    
    # >>> in create_new_env_yml:
    new_prefix = get_new_prefix(user_dir, new_env_name)
    env_file = get_yml_file(yml_path)
    new_deps = get_python_new_deps(env_file, new_py_ver)

    # update dict:
    env_file["name"] = new_env_name
    env_file["dependencies"] = new_deps
    env_file["prefix"] = new_prefix
    # <<<<
    
    #todo: create new env file
    #create_new_env_yml(yml_path, env_file, new_py_ver, yml_new_filename, env_file)
    
    return
    

In [137]:
yml_path

WindowsPath('C:/Users/catch/env_generic_minimal_test.yml')

In [130]:
python_dirs = get_python_dirs().split('\n')
python_dirs

user_dirs = get_user_dirs(python_dirs)
user_dirs
str(user_dirs[0])

['C:\\Users\\catch\\Anaconda3\\envs\\py310\\python.exe', 'C:\\Program Files\\Python310\\python.exe', 'C:\\Users\\catch\\AppData\\Local\\Microsoft\\WindowsApps\\python.exe', '']

[WindowsPath('C:/Users/catch')]

'C:\\Users\\catch'

In [134]:
# For testing, env_generic_minimal_test.yml has no python dep & prefix line is "prefix: envs_path\some_name"
yml_filename = "env_generic_minimal_test.yml"
yml_path = user_dirs[0].joinpath(yml_filename)

env_file = get_yml_file(yml_path)
env_file.keys()

dict_keys(['name', 'channels', 'dependencies', 'prefix'])

In [127]:
pp(env_file)

{'channels': ['conda-forge', 'bioconda', 'pyviz', 'anaconda', 'defaults'],
 'dependencies': ['numpy',
                  'scipy',
                  'pandas',
                  'scikit-learn',
                  'ca-certificates',
                  'certifi',
                  'openssl',
                  'matplotlib',
                  'seaborn',
                  'panel',
                  'hvplot',
                  'nodejs',
                  'json',
                  'jupyterlab',
                  'traitlets',
                  'nbformat',
                  'jupyterlab_server',
                  'jupyter_server',
                  'click'],
 'name': 'some_name',
 'prefix': 'envs_path\\some_name'}


'env_generic_minimal_fav39.yml'